In [ ]:
from utils import WindowGenerator
import numpy as np
import pandas as pd
import tensorflow as tf
import IPython
import matplotlib.pyplot as plt
import pickle

In [ ]:
# Import data
features = ['NO1_consumption', 
            'NO1_temperature', 
            'time_of_day', 
            'time_of_week', 
            'time_of_year', 
            'NO1_consumption_lag_24', 
            'NO1_temperature_lag_24', 
            'NO1_consumption_mean_24', 
            'NO1_temperature_mean_24']

train_df = pd.read_csv('./data/train.csv')[features]
test_df = pd.read_csv('./data/test.csv')[features]
val_df = pd.read_csv('./data/val.csv')[features]

## WindowGenerator

In [ ]:
# Create window generator
window = WindowGenerator(   input_width=24, 
                            label_width=1, 
                            shift=1,
                            train_df=train_df,
                            val_df=val_df,
                            test_df=test_df,
                            label_columns=['NO1_consumption'])
window

In [ ]:
# Split data into windows
# Stack three slices, the length of the total window.
example_window = tf.stack([np.array(train_df[:window.total_window_size]),
                           np.array(train_df[100:100+window.total_window_size]),
                           np.array(train_df[200:200+window.total_window_size])])

example_inputs, example_labels = window.split_window(example_window)
window.example = example_inputs, example_labels

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

In [ ]:

window.plot()

## Long Short-Term Memory model (LSTM)
#### Single-step

In [ ]:
wide_window = WindowGenerator(  input_width=24,
                                label_width=24,
                                shift=1,
                                train_df=train_df,
                                val_df=val_df,
                                test_df=test_df,
                                label_columns=['NO1_consumption'])
wide_window

In [ ]:
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=1)
])

val_performance = {}
performance = {}

print('Input shape:', wide_window.example[0].shape)
print('Output shape:', lstm_model(wide_window.example[0]).shape)

In [ ]:
# history = WindowGenerator.compile_and_fit(lstm_model, wide_window)
# IPython.display.clear_output()
# lstm_model.save('./models/lstm_single_step.keras')
# pickle.dump(window, open('./models/lstm_single_step_history.pkl', 'wb'))


In [ ]:
# Load
lstm_model = tf.keras.models.load_model('./models/lstm_single_step.keras')
history = pickle.load(open('./models/lstm_single_step_history.pkl', 'rb'))

In [ ]:
val_performance['LSTM'] = lstm_model.evaluate(wide_window.val)
performance['LSTM'] = lstm_model.evaluate(wide_window.test, verbose=0)

In [ ]:
test_window = tf.stack([np.array(test_df[:wide_window.total_window_size]),
                        np.array(test_df[100:100+wide_window.total_window_size]),
                        np.array(test_df[200:200+wide_window.total_window_size])])

test_inputs, test_labels = wide_window.split_window(test_window)

print('Test input shape:', test_inputs.shape)
print('Test label shape:', test_labels.shape)

In [ ]:
# Plot training and validation loss
# history_dict = history.history
# loss = history_dict['loss']
# val_loss = history_dict['val_loss']

# epochs = range(1, len(loss) + 1)

# plt.figure()
# plt.plot(epochs, loss, 'b', label='Training loss')
# plt.plot(epochs, val_loss, 'r', label='Validation loss')
# plt.title('Training and validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

In [ ]:
# Plot
wide_window.plot(model=lstm_model, denormalize=True)

## Long Short-Term Memory model (LSTM)
#### Multi-step

In [ ]:
OUT_STEPS = 24
multi_window = WindowGenerator(input_width=24,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               label_columns=['NO1_consumption'])

example_window = tf.stack([np.array(train_df[:multi_window.total_window_size]),
                           np.array(train_df[100:100+multi_window.total_window_size]),
                           np.array(train_df[200:200+multi_window.total_window_size])])

example_inputs, example_labels = multi_window.split_window(example_window)
multi_window.example = example_inputs, example_labels
multi_window.plot()
multi_window

In [ ]:
num_features = train_df.shape[1]
multi_lstm_model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*num_features,kernel_initializer=tf.initializers.zeros()),
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])


In [ ]:
# history = WindowGenerator.compile_and_fit(multi_lstm_model, multi_window)
# IPython.display.clear_output()
# multi_lstm_model.save('./models/lstm_multi_step.keras')
# pickle.dump(history, open('./models/lstm_multi_step_history.pkl', 'wb'))

In [ ]:
# Load
multi_lstm_model = tf.keras.models.load_model('./models/lstm_multi_step.keras')
history = pickle.load(open('./models/lstm_multi_step_history.pkl', 'rb'))

In [ ]:
multi_val_performance = {}
multi_performance = {}

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)

In [ ]:
# Plot training and validation loss
# history_dict = history.history
# loss = history_dict['loss']
# val_loss = history_dict['val_loss']

# epochs = range(1, len(loss) + 1)

# plt.figure()
# plt.plot(epochs, loss, 'b', label='Training loss')
# plt.plot(epochs, val_loss, 'r', label='Validation loss')
# plt.title('Training and validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

In [ ]:
multi_window.plot(multi_lstm_model, denormalize=True)

### Multi-step prediction on bidding area NO2

In [ ]:
features = ['NO2_consumption', 
            'NO2_temperature', 
            'time_of_day', 
            'time_of_week', 
            'time_of_year', 
            'NO2_consumption_lag_24', 
            'NO2_temperature_lag_24', 
            'NO2_consumption_mean_24', 
            'NO2_temperature_mean_24']

test_df = pd.read_csv('./data/test.csv')[features]
val_df = pd.read_csv('./data/val.csv')[features]
train_df = pd.read_csv('./data/train.csv')[features]

multi_window_no2 = WindowGenerator( input_width=24,
                                label_width=OUT_STEPS,
                                shift=OUT_STEPS,
                                label_columns=['NO2_consumption'],
                                train_df=train_df,
                                val_df=val_df,
                                test_df=test_df)

In [ ]:
example_window = tf.stack([np.array(test_df[:multi_window_no2.total_window_size]),
                            np.array(test_df[100:100+multi_window_no2.total_window_size]),
                            np.array(test_df[200:200+multi_window_no2.total_window_size])])

multi_window_no2.example = multi_window_no2.split_window(example_window)

multi_window_no2.plot(multi_lstm_model, plot_col='NO2_consumption', denormalize=True)

## Convolutional Neural Network (CNN) 

#### n-in-1-out

In [ ]:
features = ['NO1_consumption',
            'NO1_temperature',
            'time_of_day',
            'time_of_week',
            'time_of_year',
            'NO1_consumption_lag_24',
            'NO1_temperature_lag_24',
            'NO1_consumption_mean_24',
            'NO1_temperature_mean_24']

train_df = pd.read_csv('./data/train.csv')[features]
val_df = pd.read_csv('./data/val.csv')[features]
test_df = pd.read_csv('./data/test.csv')[features]

window = WindowGenerator(   input_width=24,
                            label_width=1,
                            shift=1,
                            label_columns=['NO1_consumption'])